In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gtzan-dataset-music-genre-classification/Data/features_3_sec.csv
/kaggle/input/gtzan-dataset-music-genre-classification/Data/features_30_sec.csv
/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original/disco/disco00005.png
/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original/disco/disco00057.png
/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original/disco/disco00020.png
/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original/disco/disco00072.png
/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original/disco/disco00040.png
/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original/disco/disco00043.png
/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original/disco/disco00010.png
/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original/disco/disco00071.png
/kaggle/input/gtzan-dataset-music-genre-classification/Data/image

# Description
Task 6:  Music Genre Classification Description
 
 Description:
 * Dataset (Recommended): GTZAN (Kaggle)
 * Classify songs into genres based on extracted audio features
 * Preprocess features such as MFCCs or use spectrogram images
 * Train and evaluate a multi-class model using tabular or image data
 * If image-based, use a CNN model
 
 Tools & Libraries:
 Python
 Librosa (for features)
 Scikit-learn or Keras 
 
 Covered Topics:
Audio data / CNNs |  Multi-class classification
 
 Bonus:
 Try both tabular and image-based approaches and compare results .
Use transfer learning on spectrograms

# Import Libraries

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16

2025-08-29 13:47:30.188733: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756475250.560280      77 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756475250.661108      77 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# TABULAR APPROACH (using MFCC features CSVs)

In [3]:
# Load features (3-sec segments is usually best)
csv_path = "/kaggle/input/gtzan-dataset-music-genre-classification/Data/features_3_sec.csv"
df = pd.read_csv(csv_path)

print("Dataset shape:", df.shape)
df.head()

Dataset shape: (9990, 60)


,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues


In [4]:
# Features and labels
X = df.drop(columns=["filename", "label"])
y = df["label"]

In [5]:
# Encode labels
le = LabelEncoder()
y = le.fit_transform(y)

In [6]:
# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [7]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

In [8]:
# Build a simple DNN
tab_model = models.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(len(np.unique(y)), activation="softmax")
])

tab_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

I0000 00:00:1756475268.555918      77 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1756475268.556570      77 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [9]:
# Train
history = tab_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=64)

Epoch 1/20


I0000 00:00:1756475271.813801     128 service.cc:148] XLA service 0x7bbe70011830 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1756475271.815575     128 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1756475271.815595     128 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1756475272.103592     128 cuda_dnn.cc:529] Loaded cuDNN version 90300


 75/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3307 - loss: 1.8760

I0000 00:00:1756475273.814847     128 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.3829 - loss: 1.7287 - val_accuracy: 0.6522 - val_loss: 0.9967
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6341 - loss: 1.0536 - val_accuracy: 0.7167 - val_loss: 0.8302
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6918 - loss: 0.9020 - val_accuracy: 0.7417 - val_loss: 0.7535
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7178 - loss: 0.8073 - val_accuracy: 0.7653 - val_loss: 0.6865
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7490 - loss: 0.7362 - val_accuracy: 0.7873 - val_loss: 0.6344
Epoch 6/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7602 - loss: 0.6998 - val_accuracy: 0.7848 - val_loss: 0.6113
Epoch 7/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7913 - loss: 0.6353 - val_accuracy: 0.7983 - val_loss: 0.5811
Epoch 8/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7928 - loss: 0.5953 - val_accuracy: 0.8088 - val

In [10]:
# Evaluate
y_pred = np.argmax(tab_model.predict(X_test), axis=1)
print("\nClassification Report (Tabular):")
print(classification_report(y_test, y_pred, target_names=le.classes_))

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step

Classification Report (Tabular):
              precision    recall  f1-score   support

       blues       0.86      0.88      0.87       200
   classical       0.93      0.96      0.95       199
     country       0.84      0.81      0.83       199
       disco       0.79      0.85      0.82       200
      hiphop       0.92      0.80      0.85       200
        jazz       0.87      0.91      0.89       200
       metal       0.95      0.92      0.93       200
         pop       0.91      0.82      0.86       200
      reggae       0.79      0.90      0.84       200
        rock       0.79      0.78      0.78       200

    accuracy                           0.86      1998
   macro avg       0.86      0.86      0.86      1998
weighted avg       0.86      0.86      0.86      1998



# IMAGE-BASED APPROACH (CNN on spectrograms)

In [11]:
img_dir = "/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original"

In [12]:
# Image preprocessing
img_height, img_width = 128, 128
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    img_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    subset="training",
    class_mode="categorical"
)

val_gen = datagen.flow_from_directory(
    img_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    subset="validation",
    class_mode="categorical"
)

Found 800 images belonging to 10 classes.
Found 199 images belonging to 10 classes.


# Train CNN from scratch

In [13]:
# Option A: Train CNN from scratch
cnn_model = models.Sequential([
    layers.Conv2D(32, (3,3), activation="relu", input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation="relu"),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation="relu"),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(train_gen.num_classes, activation="softmax")
])

cnn_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
# Train CNN
cnn_history = cnn_model.fit(train_gen, validation_data=val_gen, epochs=10)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 20s 545ms/step - accuracy: 0.1075 - loss: 2.4585 - val_accuracy: 0.2362 - val_loss: 2.2826
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 170ms/step - accuracy: 0.2153 - loss: 2.2516 - val_accuracy: 0.2915 - val_loss: 2.0998
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 168ms/step - accuracy: 0.2623 - loss: 2.0448 - val_accuracy: 0.3819 - val_loss: 1.9110
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 177ms/step - accuracy: 0.3581 - loss: 1.8846 - val_accuracy: 0.4573 - val_loss: 1.7922
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 183ms/step - accuracy: 0.4007 - loss: 1.7040 - val_accuracy: 0.3518 - val_loss: 1.7769
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 170ms/step - accuracy: 0.3943 - loss: 1.6979 - val_accuracy: 0.4523 - val_loss: 1.5836
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 179ms/step - accuracy: 0.5045 - loss: 1.4479 - val_accuracy: 0.4673 - val_loss: 1.6706
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 181ms/step - accuracy: 0.5573 - loss: 1.2198 - val_accuracy: 0

# Transfer Learning (VGG16 pretrained)

In [15]:
# Option B: Transfer Learning (VGG16 pretrained)
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False

tl_model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(train_gen.num_classes, activation="softmax")
])

tl_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [16]:
# Train Transfer Learning Model
tl_history = tl_model.fit(train_gen, validation_data=val_gen, epochs=5)

Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 15s 307ms/step - accuracy: 0.1368 - loss: 2.7867 - val_accuracy: 0.2010 - val_loss: 2.0008
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 175ms/step - accuracy: 0.2984 - loss: 1.9955 - val_accuracy: 0.3317 - val_loss: 1.8426
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 186ms/step - accuracy: 0.3472 - loss: 1.8237 - val_accuracy: 0.4422 - val_loss: 1.7321
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 169ms/step - accuracy: 0.3443 - loss: 1.7947 - val_accuracy: 0.4020 - val_loss: 1.7294
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 167ms/step - accuracy: 0.3731 - loss: 1.7363 - val_accuracy: 0.4372 - val_loss: 1.6608


# Results

In [17]:
print("\n✅ Tabular Model Accuracy:", history.history["val_accuracy"][-1])
print("✅ CNN Model Accuracy:", cnn_history.history["val_accuracy"][-1])
print("✅ Transfer Learning (VGG16) Accuracy:", tl_history.history["val_accuracy"][-1])


✅ Tabular Model Accuracy: 0.8618618845939636
✅ CNN Model Accuracy: 0.5175879597663879
✅ Transfer Learning (VGG16) Accuracy: 0.43718594312667847
